# Business Problem

#### Context

Dataset that used is online used car sales in Saudi Arab on syarah.com website. syarah.com is company which offer buying and selling used cars for personal or even for institutional fleets, also offer car auctions. The interaction that occurs in syarahcom is between syarah.com administration and potential buyers without a third party.

Based on [KSA Transportation](https://www.expatarrivals.com/middle-east/saudi-arabia/riyadh/getting-around-riyadh), the main transportation commonly used in Saudi Arabia is personal car because the public transportaion in incovenience especially for tourist who can't speak arabic, the gasoline in Saudi Arabia is not that expensive and the roads in Saudi Arabia are well-maintained. That's make personal transportaion the most convenience choice for people in Saudi Arabia.

Potential buyer who wants to buy a car in syarah.com is pretty easy. User only have to create an account in syarah.com. Then user can choose **New** option for a new car, or **Guaranteed Used** option for used car. Besides that, user also can choose car based on the brand, the shape of the desired model. The car's price is already determined by syarah.com, many factors affect the price of the car. And syarah.com make profit from the difference between the purchasing price syarah.com made and the selling price for potential buyer.

#### Problem Statement

syarah.com's challenge is how to set the suitable price for a car and make profit out of it financially with consindering the condition of the car, the price also can compete with other competitor and become costumer priority for buying a car especeially used car in this case.

the consideration syarah.com make to set the car's price is the car's condition, car's production year, the machine, and many more to make the price is competitive and sutiable with the quality of the car

#### Goals

Based on the issue, syarah.com need a 'tool' that can make a prediction to set a suitable price for them to market the used car. The differences in various features such as years, usage on a used car unit may increase the accuracy of predicting the price. With making this model syarah.com can make a good profit from selling used car

#### Analytic Approach

Analysis needed to be done is analyzing the dataset to create a pattern based on the existence various features, which will differentiate the price of every cars.

Next step needed to be done is to build a regression model which will support the company to set a suitable price for the cars.

#### Metric Evaluation

Metric Evaluation used are RMSE, MAE and MSE. Which RMSE is the mean value of the square root of the error, MAE is the mean absolute value of the error, while MSE is the mean square of the error. The smaller RMSE, MAE, and MAPE values, it means that the model is more accurate in predicting prices according to the limitations of the features used. 

In addtion, we can also use R-squared value. The R-squared value is used to determine how well the model can represent the overall variance of the data. The closer to 1, the more fit the model is to the observation data. However, this metric is not valid for non-linear models.

# Data Understanding

- The dataset is listing data for used car sales from syarah.com in Saudi Arabia.
- Each column represents information related to used car sales listings.

**Attributes Information**

| **Attribute** | **Data Type** | **Description** |
| --- | --- | --- |
| Type | Object | Car Type |
| Region | Object | Location Car Listing |
| Make | Object | Brand |
| Gear_Types | Object | Transmission |
| Origin | Object | Car Origin |
| Options | Object | Used Car Options |
| Year | Integer | Production Year |
| Engine_Size | Float | Engine Size |
| Mileage | Integer | Mileage |
| Negotiable | Boolean | True (if the price is no set) |
| Price | Integer | Car Price |

<br>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, PowerTransformer, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import  train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [2]:
pd.options.display.float_format='{:.2f}'.format

In [3]:
df = pd.read_csv('data_saudi_used_cars.csv')
df

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
0,Corolla,Abha,Toyota,Manual,Saudi,Standard,2013,1.40,421000,True,0
1,Yukon,Riyadh,GMC,Automatic,Saudi,Full,2014,8.00,80000,False,120000
2,Range Rover,Riyadh,Land Rover,Automatic,Gulf Arabic,Full,2015,5.00,140000,False,260000
3,Optima,Hafar Al-Batin,Kia,Automatic,Saudi,Semi Full,2015,2.40,220000,False,42000
4,FJ,Riyadh,Toyota,Automatic,Saudi,Full,2020,4.00,49000,True,0
...,...,...,...,...,...,...,...,...,...,...,...
5619,X-Trail,Jeddah,Nissan,Automatic,Saudi,Standard,2018,2.50,78794,False,68000
5620,Altima,Jeddah,Nissan,Automatic,Saudi,Semi Full,2020,2.50,58000,True,0
5621,Yukon,Dammam,GMC,Automatic,Saudi,Full,2016,5.30,48000,False,145000
5622,RX,Dammam,Lexus,Automatic,Gulf Arabic,Full,2016,2.50,81000,False,154000


# Data Preprocessing

In [4]:
df2 = df.copy()

In [33]:
datadesc = []
for i in df2.columns:
    datadesc.append([i,
                     df2[i].dtypes,
                     df2[i].isna().sum(),
                     round((df2[i].isna().sum()/len(df2)*100),2),
                     df2[i].nunique(),
                     df2[i].drop_duplicates().sample(1).values])

pd.DataFrame(datadesc,columns=['Data Features',
                               'Data Types',
                               'Null',
                               'Null Percentage',
                               'Unique',
                               'Unique Sample'])

,Data Features,Data Types,Null,Null Percentage,Unique,Unique Sample
0,Type,object,0,0.0,347,[Boxer]
1,Region,object,0,0.0,27,[Tabouk]
2,Make,object,0,0.0,58,[Škoda]
3,Gear_Type,object,0,0.0,2,[Manual]
4,Origin,object,0,0.0,4,[Other]
5,Options,object,0,0.0,3,[Full]
6,Year,int64,0,0.0,50,[1997]
7,Engine_Size,float64,0,0.0,71,[7.5]
8,Mileage,int64,0,0.0,1716,[66796]
9,Negotiable,bool,0,0.0,2,[True]


In [4]:
df2.duplicated().sum()

4

In [5]:
df2.drop_duplicates(inplace=True)

In [36]:
df2.duplicated().sum()

0

In [6]:
df2[df2['Price']==0]

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
0,Corolla,Abha,Toyota,Manual,Saudi,Standard,2013,1.40,421000,True,0
4,FJ,Riyadh,Toyota,Automatic,Saudi,Full,2020,4.00,49000,True,0
9,LS,Aseer,Lexus,Automatic,Saudi,Full,1998,4.60,300,True,0
14,Hilux,Yanbu,Toyota,Automatic,Saudi,Full,2014,2.70,209,True,0
15,Land Cruiser,Aseer,Toyota,Manual,Gulf Arabic,Semi Full,1982,4.60,340000,True,0
...,...,...,...,...,...,...,...,...,...,...,...
5610,Town Car,Jeddah,Lincoln,Automatic,Other,Full,1999,5.00,343,True,0
5614,Victoria,Makkah,Ford,Automatic,Saudi,Standard,2010,4.60,345000,True,0
5617,Yukon,Jubail,GMC,Automatic,Saudi,Semi Full,2006,5.30,300,True,0
5620,Altima,Jeddah,Nissan,Automatic,Saudi,Semi Full,2020,2.50,58000,True,0


In [7]:
df2.drop(df2[(df2['Price']==0)].index, inplace =True)

In [6]:
df2[df2['Price']==0]

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price


In [9]:
df2[df2['Mileage']==0]

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price


In [40]:
df2

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
1,Yukon,Riyadh,GMC,Automatic,Saudi,Full,2014,8.0,80000,False,120000
2,Range Rover,Riyadh,Land Rover,Automatic,Gulf Arabic,Full,2015,5.0,140000,False,260000
3,Optima,Hafar Al-Batin,Kia,Automatic,Saudi,Semi Full,2015,2.4,220000,False,42000
5,CX3,Abha,Mazda,Automatic,Saudi,Semi Full,2019,2.0,25000,False,58000
6,Cayenne S,Riyadh,Porsche,Automatic,Saudi,Full,2012,4.8,189000,False,85000
...,...,...,...,...,...,...,...,...,...,...,...
5616,CX9,Qassim,Mazda,Automatic,Saudi,Full,2020,2.0,9500,False,132000
5618,Suburban,Riyadh,GMC,Automatic,Saudi,Full,2006,5.3,86000,False,50000
5619,X-Trail,Jeddah,Nissan,Automatic,Saudi,Standard,2018,2.5,78794,False,68000
5621,Yukon,Dammam,GMC,Automatic,Saudi,Full,2016,5.3,48000,False,145000


In [47]:
df2['Type'].value_counts()

Camry           149
Land Cruiser    148
Accent          147
Hilux           118
Sonata          115
               ... 
Suvana            1
EC8               1
HS                1
A4                1
S8                1
Name: Type, Length: 320, dtype: int64

In [7]:
pd.crosstab(index=df2['Region'], columns='rata-rata', values=df2['Price'], aggfunc='mean').sort_values('rata-rata', ascending=False)

col_0,rata-rata
Region,
Dammam,93135.96
Riyadh,81109.50
Sakaka,80000.00
Qassim,79535.40
Jeddah,74252.48
Khobar,72263.96
Makkah,70723.72
Aseer,66964.61
Jazan,66220.59


In [8]:
pd.crosstab(index=df2['Type'], columns='rata-rata', values=df2['Price'], aggfunc='mean').sort_values('rata-rata', ascending=False)

col_0,rata-rata
Type,
Bentayga,850000.00
Camargue,500000.00
Ghost,465000.00
Defender,450000.00
Vanquish,400000.00
...,...
Lumina,9000.00
Optra,7916.67
D'max,7000.00


In [23]:
pd.crosstab(index=df2['Make'], columns='rata-rata', values=df2['Price'], aggfunc='mean').sort_values('rata-rata', ascending=False)

col_0,rata-rata
Make,
Rolls-Royce,476666.666667
Bentley,388750.000000
Aston Martin,290000.000000
Ferrari,280000.000000
Land Rover,237798.387097
Maserati,211666.666667
Lexus,177085.616438
Porsche,167533.333333
Mercedes,163832.369942


In [9]:
df3 = pd.crosstab(index=df['Type'], columns='count').sort_values('count',ascending=False).reset_index()

In [8]:
df3[df3['count']>200]

col_0,Type,count
0,Land Cruiser,269
1,Camry,234
2,Hilux,207


In [10]:
def type_car(x):
    if x == 'Land Cruiser':
        return x
    elif x == 'Camry':
        return x
    elif x == 'Hilux':
        return x
    else:
        return 'Others'

In [11]:
df2['Type_Car'] = df2['Type'].apply(type_car)

In [11]:
df2

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price,Type_Car
1,Yukon,Riyadh,GMC,Automatic,Saudi,Full,2014,8.0,80000,False,120000,Others
2,Range Rover,Riyadh,Land Rover,Automatic,Gulf Arabic,Full,2015,5.0,140000,False,260000,Others
3,Optima,Hafar Al-Batin,Kia,Automatic,Saudi,Semi Full,2015,2.4,220000,False,42000,Others
5,CX3,Abha,Mazda,Automatic,Saudi,Semi Full,2019,2.0,25000,False,58000,Others
6,Cayenne S,Riyadh,Porsche,Automatic,Saudi,Full,2012,4.8,189000,False,85000,Others
...,...,...,...,...,...,...,...,...,...,...,...,...
5616,CX9,Qassim,Mazda,Automatic,Saudi,Full,2020,2.0,9500,False,132000,Others
5618,Suburban,Riyadh,GMC,Automatic,Saudi,Full,2006,5.3,86000,False,50000,Others
5619,X-Trail,Jeddah,Nissan,Automatic,Saudi,Standard,2018,2.5,78794,False,68000,Others
5621,Yukon,Dammam,GMC,Automatic,Saudi,Full,2016,5.3,48000,False,145000,Others


In [12]:
df2.drop(['Type'], axis=1, inplace = True)
df2.drop(['Region'], axis=1, inplace = True)
df2.drop(['Make'], axis=1, inplace = True)
df2.drop(['Negotiable'], axis=1, inplace = True)

In [13]:
df2

,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Price,Type_Car
1,Automatic,Saudi,Full,2014,8.00,80000,120000,Others
2,Automatic,Gulf Arabic,Full,2015,5.00,140000,260000,Others
3,Automatic,Saudi,Semi Full,2015,2.40,220000,42000,Others
5,Automatic,Saudi,Semi Full,2019,2.00,25000,58000,Others
6,Automatic,Saudi,Full,2012,4.80,189000,85000,Others
...,...,...,...,...,...,...,...,...
5616,Automatic,Saudi,Full,2020,2.00,9500,132000,Others
5618,Automatic,Saudi,Full,2006,5.30,86000,50000,Others
5619,Automatic,Saudi,Standard,2018,2.50,78794,68000,Others
5621,Automatic,Saudi,Full,2016,5.30,48000,145000,Others


In [15]:
df2['Type_Car'].value_counts()

Others          3409
Camry            149
Land Cruiser     148
Hilux            118
Name: Type_Car, dtype: int64

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3824 entries, 1 to 5622
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gear_Type    3824 non-null   object 
 1   Origin       3824 non-null   object 
 2   Options      3824 non-null   object 
 3   Year         3824 non-null   int64  
 4   Engine_Size  3824 non-null   float64
 5   Mileage      3824 non-null   int64  
 6   Price        3824 non-null   int64  
 7   Type_Car     3824 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 268.9+ KB


In [14]:
num_columns = ['Year', 'Engine_Size', 'Mileage']
cat_columns = ['Gear_Type', 'Options', 'Origin', 'Type_Car']

numeric_pipeline = Pipeline([ 
    ('scaler', RobustScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('power', PowerTransformer(method='yeo-johnson'))
])

categoric_pipeline = Pipeline([
    ('encoder', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, num_columns),
    ('categoric', categoric_pipeline, cat_columns)
])

pipe_LR = Pipeline([
    ('prep', preprocessor),
    ('algo', LinearRegression()),
])

pipe_KNN = Pipeline([
    ('prep', preprocessor),
    ('algo', KNeighborsRegressor()),
])

In [15]:
num_columns = ['Year', 'Engine_Size', 'Mileage']
cat_columns = ['Gear_Type', 'Options', 'Origin', 'Type_Car']

numeric_pipeline = Pipeline([ 
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('power', PowerTransformer(method='yeo-johnson'))
])

categoric_pipeline = Pipeline([
    ('encoder', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, num_columns),
    ('categoric', categoric_pipeline, cat_columns)
])

pipe_RF = Pipeline([
    ('prep', preprocessor),
    ('algo', RandomForestRegressor()),
])

pipe_DT = Pipeline([
    ('prep', preprocessor),
    ('algo', DecisionTreeRegressor()),
])

pipe_XGB = Pipeline([
    ('prep', preprocessor),
    ('algo', XGBRegressor()),
])

In [16]:
X = df2.drop(columns='Price')
y = df2['Price']

In [50]:
X.head()

,Type,Gear_Type,Origin,Options,Year,Engine_Size,Mileage
1,Yukon,Automatic,Saudi,Full,2014,8.0,80000
2,Range Rover,Automatic,Gulf Arabic,Full,2015,5.0,140000
3,Optima,Automatic,Saudi,Semi Full,2015,2.4,220000
5,CX3,Automatic,Saudi,Semi Full,2019,2.0,25000
6,Cayenne S,Automatic,Saudi,Full,2012,4.8,189000


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)

In [18]:
list_algo = [pipe_LR, pipe_RF, pipe_DT, pipe_KNN, pipe_XGB]
for i in list_algo:
    i.fit(X_train, y_train)

In [19]:
def Eva_Matrix_DF(Model, X_train, X_test, y_train, y_test, Nama):
    y_pred_tr = Model.predict(X_train)
    R2_tr = r2_score(y_train, y_pred_tr)
    MAE_tr = mean_absolute_error(y_train, y_pred_tr)
    MSE_tr = mean_squared_error(y_train, y_pred_tr)
    RMSE_tr = np.sqrt(MSE_tr)
    y_pred_ts = Model.predict(X_test)
    R2_ts = r2_score(y_test, y_pred_ts)
    MAE_ts = mean_absolute_error(y_test, y_pred_ts)
    MSE_ts = mean_squared_error(y_test, y_pred_ts)
    RMSE_ts = np.sqrt(MSE_ts)
    data = {'Training'+ Nama : [R2_tr, MAE_tr, MSE_tr, RMSE_tr],
            'Testing' + Nama :[R2_ts, MAE_ts, MSE_ts, RMSE_ts]}
    df = pd.DataFrame(data=data, index=['R2', 'MAE', 'MSE', 'RMSE'])
    return df

In [20]:
df_LR_base = Eva_Matrix_DF(pipe_LR, X_train, X_test, y_train, y_test, 'Base LinReg')
df_RF_base = Eva_Matrix_DF(pipe_RF, X_train, X_test, y_train, y_test, 'Base Random Forest')
df_DT_base = Eva_Matrix_DF(pipe_DT, X_train, X_test, y_train, y_test, 'Base Decision Tree')
df_KNN_base = Eva_Matrix_DF(pipe_KNN, X_train, X_test, y_train, y_test, 'Base KNN')
df_XGB_base = Eva_Matrix_DF(pipe_XGB, X_train, X_test, y_train, y_test, 'Base XGBoost')

In [20]:
dfbase = pd.concat([df_LR_base, df_RF_base, df_DT_base, df_KNN_base, df_XGB_base], axis=1)
dfbase

,TrainingBase LinReg,TestingBase LinReg,TrainingBase Random Forest,TestingBase Random Forest,TrainingBase Decision Tree,TestingBase Decision Tree,TrainingBase KNN,TestingBase KNN,TrainingBase XGBoost,TestingBase XGBoost
R2,0.46,-0.76,0.93,0.74,1.00,0.50,0.64,0.68,0.95,0.72
MAE,30671.54,32817.31,9623.14,22138.80,198.91,30230.41,22705.20,24099.04,9425.38,23628.24
MSE,2816460976.22,9968768422.96,353177270.34,1492754301.54,11880258.16,2844669105.04,1868987608.40,1803953473.16,236191868.06,1605073020.73
RMSE,53070.34,99843.72,18793.01,38636.18,3446.78,53335.44,43231.79,42472.97,15368.54,40063.36


In [ ]:
np.mean(cross_val_score(pipe_RF, X,y, scoring='neg_root_mean_squared_error', cv=3))

-47607.32610644309

In [ ]:
np.mean(cross_val_score(pipe_DT, X,y, scoring='neg_root_mean_squared_error', cv=3))

-65707.31203668147

In [21]:
np.mean(cross_val_score(pipe_KNN, X,y, scoring='neg_root_mean_squared_error', cv=3))

-51032.50854028991

In [22]:
np.mean(cross_val_score(pipe_LR, X,y, scoring='neg_root_mean_squared_error', cv=3))

-62173.47306789991

In [23]:
np.mean(cross_val_score(pipe_XGB, X,y, scoring='neg_root_mean_squared_error', cv=3))

-48530.88737285921

### Hyperparameter Tuning Random Forest

In [21]:
param = {'algo__n_estimators':np.arange(1,100,1),
            'algo__max_depth':np.arange(100,1000,20),
            'algo__min_samples_leaf':np.arange(1,50,1),
            'algo__max_features':np.arange(0,20,0.5)}

In [22]:
RS_RF = RandomizedSearchCV(pipe_RF, param, cv=3, n_jobs=-1, verbose=1, scoring='r2', n_iter=500)

In [23]:
RS_RF.fit(X_train, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('numeric',
                                                                               Pipeline(steps=[('poly',
                                                                                                PolynomialFeatures(include_bias=False)),
                                                                                               ('power',
                                                                                                PowerTransformer())]),
                                                                               ['Year',
                                                                                'Engine_Size',
                                                                                'Mileage']),
                                                                              ('categoric'

In [27]:
# Cek skor dan params terbaik
print('Random Forest')
print('Best_score:', RS_RF.best_score_)
print('Best_params:', RS_RF.best_params_)

Random Forest
Best_score: 0.5480876449993285
Best_params: {'algo__n_estimators': 38, 'algo__min_samples_leaf': 6, 'algo__max_features': 1.0, 'algo__max_depth': 720}


In [24]:
# Model Random Forest
model_RF = {'RF': RandomForestRegressor(random_state=1)}

# Define model terhadap estimator terbaik
RF_tuning = RS_RF.best_estimator_

# Fitting model
RF_tuning.fit(X_train, y_train)

# Predict test set
y_pred_RF_tuning = RF_tuning.predict(X_test)

# Simpan nilai metrics RMSE, MAE & MAPE setelah tuning
rmse_RF_tuning = np.sqrt(mean_squared_error(y_test, y_pred_RF_tuning))
mae_RF_tuning = mean_absolute_error(y_test, y_pred_RF_tuning)
mape_RF_tuning = mean_absolute_percentage_error(y_test, y_pred_RF_tuning)

score_after_tuning_RF = pd.DataFrame({'RMSE': rmse_RF_tuning, 'MAE': mae_RF_tuning, 'MAPE': mape_RF_tuning}, index=model_RF.keys())
score_after_tuning_RF

,RMSE,MAE,MAPE
RF,39906.01,23074.43,1.59


### Hyperparameter Tuning XGBoost

In [25]:
# Kedalaman pohon
max_depth = list(np.arange(1, 11))

# Learning rate
learning_rate = list(np.arange(1, 100)/100)

# Jumlah pohon
n_estimators = list(np.arange(100, 201))

# Jumlah baris tiap pohon (% dari total baris train set)
subsample = list(np.arange(2, 10)/10)

# Gamma (min_impurity_decrease)
gamma = list(np.arange(1, 11)) # Semakin besar nilainya, semakin konservatif/simpel modelnya

# Jumlah feature yang digunakan untuk tiap pohon (% dari total kolom train set)
colsample_bytree = list(np.arange(1, 10)/10)

# Alpha (regularization)
reg_alpha = list(np.logspace(-3, 1, 10)) # Semakin besar nilainya, semakin konservatif/simpel modelnya


# Hyperparam space XGboost
param_xgb = {
    'algo__max_depth': max_depth, 
    'algo__learning_rate': learning_rate,
    'algo__n_estimators': n_estimators,
    'algo__subsample': subsample,
    'algo__gamma': gamma,
    'algo__colsample_bytree': colsample_bytree,
    'algo__reg_alpha': reg_alpha
}

In [26]:
RS_XGB = RandomizedSearchCV(pipe_XGB, param_xgb, cv=3, n_jobs=-1, verbose=1, scoring='r2', n_iter=500)

In [27]:
RS_XGB.fit(X_train, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('numeric',
                                                                               Pipeline(steps=[('poly',
                                                                                                PolynomialFeatures(include_bias=False)),
                                                                                               ('power',
                                                                                                PowerTransformer())]),
                                                                               ['Year',
                                                                                'Engine_Size',
                                                                                'Mileage']),
                                                                              ('categoric'

In [48]:
# Cek skor dan params terbaik
print('Random Forest')
print('Best_score:', RS_XGB.best_score_)
print('Best_params:', RS_XGB.best_params_)

Random Forest
Best_score: 0.5628667186022039
Best_params: {'algo__subsample': 0.9, 'algo__reg_alpha': 0.007742636826811269, 'algo__n_estimators': 125, 'algo__max_depth': 7, 'algo__learning_rate': 0.03, 'algo__gamma': 10, 'algo__colsample_bytree': 0.6}


In [28]:
# Model XGBoost
model_XGB = {'XGB': XGBRegressor(random_state=1)}

# Define model terhadap estimator terbaik
XGB_tuning = RS_XGB.best_estimator_

# Fitting model
XGB_tuning.fit(X_train, y_train)

# Predict test set
y_pred_XGB_tuning = XGB_tuning.predict(X_test)

# Simpan nilai metrics RMSE, MAE & MAPE setelah tuning
rmse_XGB_tuning = np.sqrt(mean_squared_error(y_test, y_pred_XGB_tuning))
mae_XGB_tuning = mean_absolute_error(y_test, y_pred_XGB_tuning)
mape_XGB_tuning = mean_absolute_percentage_error(y_test, y_pred_XGB_tuning)

score_after_tuning_XGB = pd.DataFrame({'RMSE': rmse_XGB_tuning, 'MAE': mae_XGB_tuning, 'MAPE': mape_XGB_tuning}, index=model_XGB.keys())
score_after_tuning_XGB

,RMSE,MAE,MAPE
XGB,37912.42,22107.35,1.58


In [29]:
pd.concat([score_after_tuning_RF,score_after_tuning_XGB])

,RMSE,MAE,MAPE
RF,39906.01,23074.43,1.59
XGB,37912.42,22107.35,1.58


### **Conclusion**

The Evaluation Metrics used in the model are R2, MAE, MSE, RMSE. Based on the value of the Evaluation Metric, the best value is found in the `Random Forest` and `XGBoost` models with the assessment carried out on the RMSE testing number:

| **Evaluation Matrix** | **Random Forest Base** | **XGBoost Base** |
| --- | --- | --- |
| RMSE | 38636.18 | 40063.36 |

Based on these results, Hyperparameter Tuning was then performed on the two models which resulted in the following RMSE results:

| **Evaluation Matrix** | **Random Forest Tuned** | **XGBoost Tuned** |
| --- | --- | --- |
| RMSE | 39906.01 | 37912.42 |

From these results, it is concluded that the XGBoost model is the best model with the lowest RMSE value after Hyperparameter Tuning is performed

These results are not that good because many features are dropped and also the number of iterations when tuning is determined because it is limited and adapted to the device used.

### **Recommendation**

Things that can be done to develop the model to be even better, such as:

If possible, analyze the features and check the corellation for every features with the target, in this case is the price. In addition, adding and removing of features also affects the results of the model because the more complex it is, the more detailed the model will be. Try using other models that are more complex also affects  the results of the model. However, if the amount of data and features is still like this dataset, it will most likely not change the results significantly.

In addition, try creating Machine Learning models in the form of classification, for example making models to determine whether the car to be sold is profitable or not by determining the price of the car to be sold, the showroom owner also knows the car is profitable or not.

### **Save Model**

In [32]:
import pickle
pickle.dump(RS_XGB, open('BestModel.pkl', 'wb'))

### **Load Model**

In [ ]:
Model = joblib.load('BestModel.pkl')